<h1> Yandex.Afisha project.</h1>

<h2> Project description</h2>

<p>You've done beautifully in the Practicum course, and you've been offered an internship in the analytical department at Yandex.Afisha. Your first task is to help optimize marketing expenses.</p>

<p>You have:
Server logs with data on Yandex.Afisha visits from June 2017 through May 2018
Dump file with all orders for the period
Marketing expenses statistics</p>

<p>You are going to study:
How people use the product
When they start to buy
How much money each customer brings
When they pay off</p>

**Step 1. Download the data and prepare it for analysis**

Store the data on visits, orders, and expenses in variables. Optimize the data for analysis. Make sure each column contains the correct data type.

**Step 2. Make reports and calculate metrics**

**Product**

- How many people use it every day, week, and month?
- How many sessions are there per day? (One user might have more than one session.)
- What is the length of each session?
- How often do users come back?

**Sales**

- When do people start buying? (In KPI analysis, we're usually interested in knowing the time that elapses between registration and conversion — when the user becomes a customer. For example, if registration and the first purchase occur on the same day, the user might fall into category Conversion 0d. If the first purchase happens the next day, it will be Conversion 1d. You can use any approach that lets you compare the conversions of different cohorts, so that you can determine which cohort, or marketing channel, is most effective.)
- How many orders do they make during a given period of time?
- What is the average purchase size?
- How much money do they bring? (LTV)

**Marketing**
- How much money was spent? Overall/per source/over time
- How much did customer acquisition from each of the sources cost?
- How worthwhile where the investments? (ROI)


**Step 3. Write a conclusion: advise marketing experts how much money to invest and where.**
- What sources/platforms would you recommend? Back up your choice: what metrics did you focus on? Why? What conclusions did you draw after finding the metric values?

<h2>Description of the data</h2>

The visits table (server logs with data on website visits):

- Uid — user's unique identifier
- Device — user's device
- Start Ts — session start date and time
- End Ts — session end date and time
- Source Id — identifier of the ad source the user came from
- All dates in this table are in YYYY-MM-DD format.

The orders table (data on orders):

- Uid — unique identifier of the user making an order
- Buy Ts — order date and time
- Revenue — Yandex.Afisha's revenue from the order
- The costs table (data on marketing expenses):
- source_id — ad source identifier
- dt — date
- costs — expenses on this ad source on this day

# Step 1. Download the data and prepare it for analysis


In [ ]:
#!pip install -q plotly==5.5.0

In [ ]:
#pip install -U sidetable

In [ ]:
import pandas as pd
import numpy as np
import sidetable
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

In [ ]:
visits = pd.read_csv('/datasets/visits_log_us.csv')
orders = pd.read_csv('/datasets/orders_log_us.csv')
costs = pd.read_csv('/datasets/costs_us.csv')

Let's look at our visits data


In [ ]:
visits.head()

Let's look at our visits data.
We have the device, end, and the start of the session, user id, and source id.
Let's look at the info.


In [ ]:
visits.info()

We can see that a device is an object, but it can be a category data type.
The End Ts and Start Ts also can be changed to DateTime.
Let's look at missing values.


In [ ]:
visits.stb.missing()

No missing values. Let's look at the described method.

In [ ]:
visits.describe()

In [ ]:
visits.stb.freq(['Device'])

There are two types of devices: desktop and touch.
73% of the visitors come from the desktop, and the rest come from the touch.

Let's look at the other databases with the same step as we did on the visits table.

In [ ]:
orders.head()

We have Buy Ts as the date of orders, order revenue, and user id.

In [ ]:
orders.info()

We need to change Buy Ts to date time.

In [ ]:
orders.stb.missing()

In [ ]:
orders.describe()

In [ ]:
orders.stb.missing()

In [ ]:
costs.head()

In [ ]:
costs.info()

We need to change dt to DateTime.


In [ ]:
costs.stb.missing()

In [ ]:
costs.describe()

Let's change data types on the visitor's table and give the columns better names.


In [ ]:
visits.head(1)

In [ ]:
visits['End Ts'] = pd.to_datetime(visits['End Ts'])

In [ ]:
visits['Start Ts'] = pd.to_datetime(visits['Start Ts'])

In [ ]:
visits['Device'] = visits['Device'].astype('category')

In [ ]:
visits.columns = ['Device', 'session_end', 'Source_Id', 'session_start', 'Uid']

In [ ]:
visits.head(1)

In [ ]:
visits.info()

In [ ]:
visits.head(1)

Same with the order table.

In [ ]:
orders.head(1)

In [ ]:
orders['Buy Ts'] = pd.to_datetime(orders['Buy Ts'])

In [ ]:
orders = orders.rename(columns={"Buy Ts" : "order_date"})

In [ ]:
orders.info()

Same with costs table.


In [ ]:
costs.head(1)

In [ ]:
costs['dt'] = pd.to_datetime(costs['dt'])

In [ ]:
costs=costs.rename(columns={'dt': 'cost_date'})

In [ ]:
costs.info()

The data is clean and ready for analysis.


# Step 2. Make reports and calculate metrics:


**1) Product**

**How many people use it every day, week, and month?**

Let's create additional date parameters to help us in the cohort analysis.

In [ ]:
visits.head()

In [ ]:
visits['session_day'] = visits['session_start'].astype('datetime64[D]')
visits['session_week'] = visits['session_start'].astype('datetime64[W]')
visits['session_month'] = visits['session_start'].astype('datetime64[M]')

Let's create DAU from the visits table.


In [ ]:
dau=visits.groupby(visits['session_day'])['Uid'].nunique().reset_index()
dau.head()

lets plot the data of the DAU

In [ ]:
fig = px.line(dau,x='session_day', y='Uid', title='DAU',
             labels=dict(session_day="time", Uid="num of DAU")
             )
#adding reference line with average DAU over time
fig.add_hline(y=dau['Uid'].mean(),line_dash="dash", line_color="purple", annotation_text="average DAU",
             annotation_position="top left")
fig.show()


In [ ]:
dau['Uid'].mean()

The average DAU is 907.
Before Oct 2017, the  DAU was below average, but between Oct 2017 to Jan 2018, there was a spike in DAU.
We can notice the influence of seasonality and Black Friday.

Let's look at the weekly active users.


In [ ]:
wau=visits.groupby(visits['session_week'])['Uid'].nunique().reset_index()
wau.head()

In [ ]:
fig = px.line(wau,x='session_week', y='Uid', title='WAU',
             labels=dict(session_week="time", Uid="num of WAU")
             )
fig.add_hline(y=wau['Uid'].mean(),line_dash="dash", line_color="purple", annotation_text="average DAU",
             annotation_position="top left")
fig.show()

In [ ]:
wau['Uid'].mean()

The average weekly user is 5724.
Until Oct 2017, the WAU was below the mean, but it was above the mean between 0ct 2017 to April 2018 and then fell below the average.

Let's look at the monthly active users.

In [ ]:
mau=visits.groupby(['session_month'])['Uid'].nunique().reset_index()
mau.head()

In [ ]:
fig = px.line(mau,x='session_month', y='Uid', title='MAU',
             labels=dict(session_month="time", Uid="num of MAU")
             )
fig.add_hline(y=mau['Uid'].mean(),line_dash="dash", line_color="purple", annotation_text="average DAU",
             annotation_position="top left")

fig.show()

In [ ]:
mau['Uid'].mean()

The MAU is 23228.
Until the third quarter of September, the MAU was below average, but it has maintained above average since then.

**How many sessions are there per day? (One user might have more than one session.)**

In [ ]:
s_per_day=visits.groupby(visits['session_start']).agg({'Uid': 'nunique'}).reset_index()
s_per_day.head()

In [ ]:
fig = px.line(s_per_day,x='Uid', y='session_start', title='session per day',
             labels=dict(session_start="time", Uid="num of session")
             )
fig.show()

We usually see no more than five sessions per day from the visual. But it's a little hard to understand it.
Let's use the describe method to get some context.

In [ ]:
s_per_day['Uid'].value_counts()

In [ ]:
s_per_day.stb.freq(['Uid'])

60% made one session, 26% made two sessions, 9.5% made 3 sessions.
users who did more than three sessions are less than 5% of our data.

In [ ]:
s_per_day.describe()

There was 224303 session in our data with an average session of 1.6.
Most of the visits were one session, and the maximum session per user was 19.


**What is the length of each session?**

In [ ]:
visits.head(1)

Let's create a duration column in our visits column in seconds.

In [ ]:
visits['duration'] = (visits['session_end'] - visits['session_start']).dt.seconds

In [ ]:
visits.head(1)

In [ ]:
d_per_s = visits.groupby(['session_start'])['duration'].sum().reset_index()
d_per_s.head()

In [ ]:
fig = px.line(d_per_s,x='session_start', y='duration', title='duration per session',
             labels=dict(session_start="time", duration="duration in sec")
             )
fig.show()

It is hard to understand what is going on in this visual. Let's look at the described method and histogram.

In [ ]:
d_per_s['duration'].plot(kind='hist', bins=70)
plt.xlabel('duration')
plt.show()

Hard to see from the histogram the exact picture, but we can see that there is no more than 10000 seconds duration.
We can also see that most of the data is closer to 500 - 1000 sec, but that is hard to determine. let's look at the describe()

In [ ]:
d_per_s['duration'].describe()

The avg duration per session is 1031 sec. The median duration is 540. but the mean affected a lot by outliers like 0's(a user that might be got by mistake) and primarily by high duration period like the max: 84480 sec. (People who forgot their computer open.)

Let's remove 0's from this column. We will not remove extremely high duration because we dont really know in what duration a user didn't use his device.

In [ ]:
d_per_s = d_per_s[d_per_s['duration'] != 0]

In [ ]:
d_per_s['duration'].describe()

The mode of a set of data values is the value that appears most often. It is the value at which the data is most likely to be sampled.
Let's look at the mode.

In [ ]:
asl = d_per_s['duration'].mode()
asl

In [ ]:
60 / 60

The average session length is 1 minutes.

**How often do users come back?**

We need to calculate the retention rate. From here, we will create a monthly cohort and calculate each month's retention rate. But first, let's find our first session for each user and merge it back to the original data.

In [ ]:
visits.head(1)

In [ ]:
min_visit = visits.groupby(['Uid'])['session_start'].min().reset_index()
min_visit.columns = ['Uid', 'first_session']
min_visit.head()

In [ ]:
visits = visits.merge(min_visit, how='inner', on=['Uid'])
visits.head()

Let's create a  monthly cohort column from each first_session and calculate the age of each cohort.

In [ ]:
visits['cohort'] = visits['first_session'].astype('datetime64[M]')
visits['age'] = ((pd.to_datetime(visits['session_start']) - pd.to_datetime(visits['first_session'])) / np.timedelta64(1,'M'))\
                                                                        .round().astype('int')
visits.head()

Now let's calculate the retention rate for each cohort with a pivot table.


In [ ]:
cohorts = visits.pivot_table(index='cohort', columns='age', values='Uid', aggfunc='nunique').fillna(0)
cohorts

Now we have the number of unique users in each cohort. The retention rate is in %; let's calculate it.

In [ ]:
retention=cohorts.iloc[:,0:].div(cohorts[0], axis=0)
retention

Let's visualize it with a heatmap.

In [ ]:
# the first line just formats cohort into a str for a nicer output
plt.figure(figsize=(13, 9))
retention.index=retention.index.astype(str)
sns.heatmap(retention, annot=True, fmt='.1%', linewidths=1, linecolor='grey',  vmax=0.1, cbar_kws= {'orientation': 'horizontal'} 
            ).set(title = 'Retention Rate')

plt.show()

For the first cohort, we see a retention rate of 74% after one month, then a decrease in month two but then an increase again.
Maybe it is because of the holiday season.
Cohorts from 06 - 11 in 2017 had a retention rate of more than 70% after the first month except month 07.
The cohorts in 2017 perform much better than the cohorts in 2018 regarding retention rate, but we need to remember that these are different months.

**2) Sales**

**When do people start buying? (In KPI analysis, we're usually interested in knowing the time that elapses between registration and conversion — when the user becomes a customer. For example, if registration and the first purchase occur on the same day, the user might fall into category Conversion 0d. If the first purchase happens the next day, it will be Conversion 1d. You can use any approach that lets you compare the conversions of different cohorts, so that you can determine which cohort, or marketing channel, is most effective.)**

First, let's find the first order.

In [ ]:
orders.head(1)

We will group the user id and take the first order date of each user. Then we will merge it again with our original data.

In [ ]:
first_order = orders.groupby(['Uid'])['order_date'].min().reset_index()
first_order.columns = ['Uid', 'first_order']
first_order.head()

Let's merge the first order back to the order data.

In [ ]:
orders = orders.merge(first_order, on=['Uid'], how='left')
orders.head()

Now that we have our first order let's merge the first session into the order table.

In [ ]:
orders = orders.merge(min_visit, on=['Uid'], how='left')
orders.head()

Let's calculate the average conversion time and plot it.

In [ ]:
orders['conversion']=((pd.to_datetime(orders['first_order'])-pd.to_datetime(orders['first_session']))/np.timedelta64(1,'D')).astype('int')

In [ ]:
fig = px.histogram(orders, x="conversion",nbins=30)
fig.show()

In [ ]:
orders['conversion'].describe()

We have a lot of 0's in the table.
This is because not every session becomes a conversion.

In [ ]:
orders['conversion'].value_counts()

Let's remove people who didn't convert.

In [ ]:
conversion = orders[orders['conversion'] != 0]
conversion.head()

In [ ]:
fig = px.histogram(conversion, x="conversion",nbins=30)
fig.show()

In [ ]:
conversion['conversion'].describe()

The data is skewed to the right, which dramatically affects our average. Let's take the median.

In [ ]:
conversion['conversion'].median()

So, on average it takes 21 days to convert for a user. But how many users converted at all? 

In [ ]:
print(' The overall conversion is {:.1%}'.format(orders['Uid'].nunique()/visits['Uid'].nunique()))

**How many orders do they make during a given period of time?**

We need to create cohorts for the orders.
First, let's create the date of each user's first order month to define our cohorts. 

First, let's create the date of each user's first order month to define our cohorts. 

In [ ]:
orders['first_order_month'] = orders['first_order'].astype('datetime64[M]')
orders.head()

Let's create the month of each order.

In [ ]:
orders['month'] = orders['order_date'].astype('datetime64[M]')

In [ ]:
cohort_sizes = orders.groupby('first_order_month').agg({'Uid': 'nunique'}).reset_index()
cohort_sizes.columns = ['first_order_month', 'cohort_size']
cohort_sizes.head()

In [ ]:
#calculating number of orders for cohort and month
cohort_orders=orders.groupby(['first_order_month','month'])['Revenue'].count().reset_index()
cohort_orders.columns=['first_order_month','month','orders']
#merge cohort_orders with cohort size
cohort_orders=cohort_orders.merge(cohort_sizes,on=['first_order_month'])
cohort_orders['age_month'] = ((cohort_orders['month'] - cohort_orders['first_order_month']) / np.timedelta64(1,'M')).round()
cohort_orders['orders_per_buyer']=cohort_orders['orders']/cohort_orders['cohort_size']
cohort_orders.head()
 

In [ ]:
cohort_piv=cohort_orders.pivot_table(
    index='first_order_month', 
    columns='age_month', 
    values='orders_per_buyer', 
    aggfunc='sum'
)

cohort_piv.round(2).fillna('')

In [ ]:
plt.figure(figsize=(13, 9))
sns.heatmap(cohort_piv, annot=True, fmt='.1%', linewidths=1, linecolor='grey',  vmax=0.1, cbar_kws= {'orientation': 'horizontal'} 
            ).set(title = 'orders per user')
plt.xlabel("age")
plt.ylabel("cohorts")
plt.show()

The first month of each cohort is the best. The first cohorts 06-2017  he's the best performance, and we can see even a rise in order per user after three months and after seven months a decrease.
The 08-2017 and 09-2017 also perform well compared to other cohorts.

**What is the average purchase size?**


In [ ]:
avg_check=orders.groupby(['Uid'])['Revenue'].sum().reset_index()
avg_check = avg_check[avg_check['Revenue'] != 0]
avg_check.plot(kind='hist', x='Uid', y='Revenue')
plt.show()

Hard to understand from the histogram. let's use the describe()

In [ ]:
avg_check['Revenue'].describe()

In [ ]:
avg_check['Revenue'].mode()

the average purchase size is 2.44

**Let's look at the avg order size for each cohort month**

In [ ]:
avg_cohort=orders.groupby(['first_order_month','month'])['Revenue'].mean().reset_index()
avg_cohort['age_month'] = ((avg_cohort['month'] - avg_cohort['first_order_month']) / np.timedelta64(1,'M')).round()
avg_cohort.head()

In [ ]:
avg_cohort_piv=avg_cohort.pivot_table(
    index='first_order_month', 
    columns='age_month', 
    values='Revenue', 
    aggfunc='mean'
)

avg_cohort_piv.round(2).fillna('')

**How much money do they bring? (LTV)**

In [ ]:
#get the revenue per cohort in each month
ltv_cohort=orders.groupby(['first_order_month','month'])['Revenue'].sum().reset_index()
ltv_cohort.columns = ['first_order_month','month','revenue']
#merge with the cohort size
ltv_cohort=ltv_cohort.merge(cohort_sizes,on=['first_order_month'])
ltv_cohort['age']=((ltv_cohort['month'] - ltv_cohort['first_order_month']) / np.timedelta64(1,'M')).round()
ltv_cohort['ltv']=ltv_cohort['revenue']/ltv_cohort['cohort_size']
ltv_cohort

In [ ]:
ltv_cohort_piv=ltv_cohort.pivot_table(
    index='first_order_month', 
    columns='age', 
    values='ltv', 
    aggfunc='sum'
).cumsum(axis=1)

In [ ]:
plt.figure(figsize=(13, 9))
ltv_cohort_piv.index=ltv_cohort_piv.index.astype(str)
sns.heatmap(ltv_cohort_piv, annot=True, fmt='.2f', linewidths=1, linecolor='grey', cbar_kws= {'orientation': 'horizontal'} 
            ).set(title ='LTV')
plt.xlabel("age")
plt.ylabel("cohorts")
plt.show()

LTV is always going up. The two cohorts that really stood out are 09-2017 and 06-2017.

**3) Marketing**

**How much money was spent? Overall/per source/over time**

To answer this question we just need **costs** table. Group costs by source and visualize it.

In [ ]:
costs.head()

In [ ]:
#sorting cost for futher visualization
costs=costs.sort_values(by=['cost_date','source_id'])
costs.head()

In [ ]:
print('Total marketing cost is {}'.format(costs['costs'].sum()))

**Let's see what how it's split across sources**

In [ ]:
marketing_per_source=costs.groupby(['source_id'])['costs'].sum().reset_index()
marketing_per_source.sort_values(by='costs', ascending=False).head()

In that exact order, the top 5 costly sources in terms of ad spend are 3, 4, 5, 2, 1.

In [ ]:
#plotting costs over time
import plotly.express as px

fig = px.line(costs, x="cost_date", y="costs", color='source_id', title='Cost per source')
fig.show()

In this visual, we can see that each source behaves the same over time in terms of cost. Source 3 leads the dance here.

**How much did customer acquisition from each of the sources cost?**


For customer acquistion we will calculate it based on **customers** not visitors. Let's do it by month. We need to group costs and customers by month then.

In [ ]:
costs['costs_month'] = costs['cost_date'].astype('datetime64[M]')

In [ ]:
marketing_by_month=costs.groupby(['costs_month'])['costs'].sum().reset_index()
marketing_by_month.head()

In [ ]:
buyers_per_moth=orders.groupby(['first_order_month'])['Uid'].nunique().reset_index()
buyers_per_moth.columns=['costs_month','buyers']
buyers_per_moth.head()

In [ ]:
CAC_per_month=marketing_by_month.merge(buyers_per_moth,how='left',on=['costs_month'])
CAC_per_month['CAC']=CAC_per_month['costs']/CAC_per_month['buyers']
CAC_per_month.head()

In [ ]:
fig = px.line(CAC_per_month, x="costs_month", y="CAC", title='CAC')
fig.show()

It looks like the CAC starts around 9, and for starters, he goes up, but it decreases dramatically in the end.


#### CAC per source

In order to calculate CAC per source we need to know the original source of the user. If we check our original dataset we will see that users can come from different sources

We look at orders table because even if the user didn't buy smth, he still came from the  first source

In [ ]:
visits.head(1)

In [ ]:
visits.groupby(['Uid'])['Source_Id'].nunique().value_counts()

Not all users have one source, let's define the first that they came from as the main source.

In [ ]:
first_source=visits.sort_values('Source_Id').groupby('Uid').first()['Source_Id'].reset_index()
first_source.columns=['Uid','first_source']
first_source.head()

In [ ]:
orders=orders.merge(first_source, on=['Uid'], how='left')
orders.head()

Let's see how CAC was changing for each source across time

In [ ]:
costs.head(1)

In [ ]:
marketing_by_month_source=costs.groupby(['costs_month','source_id'])['costs'].sum().reset_index()
marketing_by_month_source.head()

In [ ]:
buyers_per_moth_source=orders.groupby(['first_order_month','first_source'])['Uid'].nunique().reset_index()
buyers_per_moth_source.columns=['costs_month','source_id','buyers']
buyers_per_moth_source.head()

In [ ]:
#actual cac calculations
CAC_per_month_source=marketing_by_month_source.merge(buyers_per_moth_source,how='left',on=['costs_month','source_id'])
CAC_per_month_source['CAC']=CAC_per_month_source['costs']/CAC_per_month_source['buyers']
CAC_per_month_source.head()

In [ ]:
#plotting cac dynamics
fig = px.line(CAC_per_month_source, x="costs_month", y="CAC",color='source_id',title='CAC')
fig.show()

It seems like source one is not utilizing enough. We sow that he is only number 5 in total cost per source, but in terms of price per acquisition source, one is the cheapest, and we should put more money on it.

How worthwhile where the investments? (ROI)

**In genereal, ROI=LTV/CAC**
    <div>
We already have calculations on CAC per month and we have info on ltv in ltv_cohort. So let's merge and work it out!

In [ ]:
CAC_per_month_ROI=CAC_per_month[['costs_month','CAC']]
CAC_per_month_ROI.columns=['first_order_month','CAC']
ROI=ltv_cohort.merge(CAC_per_month_ROI,on=['first_order_month'],how='left')
ROI.head()

In [ ]:
ROI['ROI']=ROI['ltv']/ROI['CAC']
roi_piv = ROI.pivot_table(
    index='first_order_month', columns='age', values='ROI', aggfunc='mean'
).cumsum(axis=1).round(2) 

In [ ]:
plt.figure(figsize=(13, 9))
roi_piv.index=roi_piv.index.astype(str)
sns.heatmap(roi_piv, annot=True, fmt='.2f', linewidths=1, linecolor='grey', cbar_kws= {'orientation': 'horizontal'} 
            ).set(title ='ROI per cohort')
plt.show()

We can see that the first cohort closed with the second higher ROI; the 09-2017 also performed well and finished first. The cohort of 02-2018 ended with the lowest ROI. 

In [ ]:
ltv_per_source=orders.groupby(['first_source'])['Uid','Revenue'].agg({'Uid':'nunique','Revenue':'sum'}).reset_index()
ltv_per_source.columns=['source_id','buyers','revenue']
ltv_per_source['ltv']=ltv_per_source['revenue']/ltv_per_source['buyers']
ltv_per_source


In [ ]:
roi_per_source=marketing_per_source.merge(ltv_per_source,on=['source_id'])
roi_per_source['cac']=roi_per_source['costs']/roi_per_source['buyers']
roi_per_source['romi']=roi_per_source['ltv']/roi_per_source['cac']
roi_per_source

**I have already done it, but there is a problem with the jupyter hub it got data lost. My final task and conclusion were lost /: 
I did it again, though.**

**version 2: I have already done it, but there is a problem with the jupyter hub it got data lost. My final task and conclusion were lost /: 
I did it again, though.**

**this is the third time /: I have already done it, but there is a problem with the jupyter hub it got data lost. My final task and conclusion were lost /: 
I did it again, though.**

In [ ]:
fig = px.bar(roi_per_source, x='source_id', y='romi', labels={
                     "source_id": "source", 'romi': 'return on marketing invesment'}, title='ROI PER SOURCE')
fig.update_xaxes(type='category')
fig.show()

We see that source one is the winner in terms of ROI. Also, he is only the 5th expensive source, and he has the cheapest cost per acquisition. The company should start pumping more money into this channel, and maybe it is a good idea to take it from source 3. also, source number one has a good potential in terms of ROI and CAC.

# Step 3. Write a conclusion: advise marketing experts how much money to invest and where.

We have come a long way since the mission was put on our desk.


**Step 1. Download the data and prepare it for analysis**

- We store the data on a variable.
- We learned about our data.
- We checked for duplicated values.
- We change data types.
- We change the names of the columns for a better understanding.

**Step 2. Make reports and calculate metrics:**

- We checked daily active users and found The average DAU is 907.


- Before Oct 2017, the  DAU was below average, but between Oct 2017 to Jan 2018, there was a spike in DAU. 


- We can notice the influence of seasonality and Black Friday.


- The average weekly user is 5724.


- Until Oct 2017, the WAU was below the mean, but it was above the mean between 0ct 2017 to April 2018 and then fell below the average.


- The MAU is 23228.


- Until the third quarter of September, the MAU was below average, but it has maintained above average since then.


- 60%  of users made one session, 26% made two sessions, 9.5% made three sessions.


- Users who did more than three sessions are less than 5% of our data.


**- There was 224303 session in our data with an average session of 1.6.**


- Most of the visits were one session, and the maximum session per user was 19.


- The avg duration per session is 1031 sec. The median duration is 540. but the mean affected a lot by outliers like 0's(a user that might be got by mistake) and primarily by high duration period like the max: 84480 sec. (People who forgot their computer open.)


- The average session length is 1 minute.


**- On average, it takes 21 days to convert for a user.**


**- The overall conversion is 16.0%**


- The first month of each cohort is the best. The first cohorts 06-2017  he's the best performance, and we can see even a rise in order per user after three months and after seven months a decrease.


- The 08-2017 and 09-2017 also perform well compared to other cohorts.

**- The average purchase size is 2.44.**

- LTV is always going up. However, the two cohorts that stood out were 09-2017 and 06-2017.


**- The total marketing cost is 329131.62**


- Each source behaves the same over time in terms of cost. Source 3 leads the dance.


**- CAC starts around 9, and for starters, he goes up, but it decreases dramatically in the End.**


**- Source one is not utilized enough. We sow that he is only number 5 in total cost per source, but in terms of price per acquisition source, one is the cheapest, and we should put more money on it.**


**- I recommend taking the budget from source three and investing more in source one.**


**- The first cohort closed with the second higher ROI; the 09-2017 also performed well and finished first. Unfortunately, the cohort of 02-2018 ended with the lowest ROI.**


**- We see that source one is the winner in terms of ROI. Also, he is only the 5th expensive source, and he has the cheapest cost per acquisition. The company should start pumping more money into this channel, and maybe it is a good idea to take it from source 3. also, source number two has a good potential in terms of ROI and CAC.**
